# Install dependencies

In [ ]:
%pip install -q pandas pyarrow numpy scikit-learn matplotlib seaborn nltk joblib tqdm

# 1) Setup: paths and helpers

In [ ]:
import os, re, json, math, warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, List

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

import joblib

warnings.filterwarnings("ignore")
sns.set(style="whitegrid")
plt.rcParams["figure.dpi"] = 120

# Paths (works whether you run from notebooks/ or project root)
PROJ = Path.cwd().resolve().parents[0] if Path.cwd().name == 'notebooks' else Path.cwd()
DATA = PROJ / "data"
RAW = DATA / "raw"
PROC = DATA / "processed"
MODELS = PROJ / "models"
REPORTS = PROJ / "reports"
FIGS = REPORTS / "figures"
for p in [RAW, PROC, MODELS, REPORTS, FIGS]:
    p.mkdir(parents=True, exist_ok=True)

def timestamp():
    return datetime.utcnow().strftime("%Y%m%d_%H%M%S")

def latest_file(folder: Path, pattern="*.parquet"):
    files = sorted(folder.glob(pattern), key=lambda p: p.stat().st_mtime)
    return files[-1] if files else None

RUN_ID = timestamp()
RUN_ID

# 2) Parameters

In [ ]:
# Mode: "auto" | "supervised" | "unsupervised"
MODE = "auto"

# Column names
TEXT_COL_CANDIDATES = ["text_clean", "text"]
LABEL_COLUMN = "label"  # if present, used in supervised mode

# Supervised params
TEST_SIZE = 0.2
RANDOM_STATE = 42
NGRAM_RANGE = (1, 2)
MAX_FEATURES = 50000
USE_CLASS_WEIGHT_BALANCED = True

# Unsupervised (VADER) params
VADER_ONLY_ENGLISH = True        # keep only lang == "en" before VADER
VADER_POS_THRESHOLD = 0.05       # >= => "pos"
VADER_NEG_THRESHOLD = -0.05      # <= => "neg"

# Optional transformer inference (set True to run)
USE_TRANSFORMER = False
TRANSFORMER_MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # English
# For multilingual, consider: "cardiffnlp/twitter-xlm-roberta-base-sentiment"
BATCH_SIZE = 32  # for transformer inference

# Artifacts naming base
ARTI_BASE = f"sentiment_{RUN_ID}"

# 3) Load processed dataset

In [ ]:
proc_path = latest_file(PROC, "*.parquet")
if not proc_path:
    raise FileNotFoundError("No processed parquet found in data/processed. Run 02_data_cleaning first.")

print(f"Loading processed snapshot: {proc_path}")
df = pd.read_parquet(proc_path)

# Pick text column
TEXT_COL = None
for c in TEXT_COL_CANDIDATES:
    if c in df.columns:
        TEXT_COL = c
        break
if TEXT_COL is None:
    raise ValueError(f"None of the expected text columns found: {TEXT_COL_CANDIDATES}")

# Parse time if available
if "created_at" in df.columns:
    df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")

print("Rows:", len(df), "| Text column:", TEXT_COL)
if LABEL_COLUMN in df.columns:
    print("Label column found:", LABEL_COLUMN, "| unique:", df[LABEL_COLUMN].dropna().unique()[:10])

# 4) Helpers for labels and plots

In [ ]:
def standardize_label(x):
    """
    Map incoming labels to {'neg','neu','pos'} if possible.
    Rules:
      - numeric > 0 => pos; < 0 => neg; == 0 => neu
      - string aliases handled (case-insensitive)
    """
    if pd.isna(x):
        return np.nan
    # numeric
    try:
        xv = float(x)
        if xv > 0: return "pos"
        if xv < 0: return "neg"
        return "neu"
    except:
        pass
    s = str(x).strip().lower()
    pos_alias = {"pos","positive","+","p","good","favorable","favourable","yes","true","1","👍","🙂","😊"}
    neg_alias = {"neg","negative","-","n","bad","unfavorable","unfavourable","no","false","-1","👎","🙁","😞"}
    neu_alias = {"neu","neutral","0","neut","meh","ok","mixed"}
    if s in pos_alias: return "pos"
    if s in neg_alias: return "neg"
    if s in neu_alias: return "neu"
    # fallback: unknown -> NaN
    return np.nan

def plot_confusion(cm, labels, title, path):
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=labels, yticklabels=labels)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(path, dpi=150)
    plt.show()
    print("Saved:", path)

# 5) Decide mode 

In [ ]:
resolved_mode = MODE
if MODE == "auto":
    if LABEL_COLUMN in df.columns:
        y_raw = df[LABEL_COLUMN].copy()
        y_std = y_raw.map(standardize_label)
        if y_std.notna().sum() >= 50 and y_std.dropna().nunique() >= 2:
            resolved_mode = "supervised"
        else:
            resolved_mode = "unsupervised"
    else:
        resolved_mode = "unsupervised"

print("Resolved MODE:", resolved_mode)

# 6) Supervised baseline

In [ ]:
if resolved_mode == "supervised":
    # Standardize and filter labels
    y = df[LABEL_COLUMN].map(standardize_label)
    mask = y.notna()
    X_text = df.loc[mask, TEXT_COL].astype(str)
    y = y.loc[mask]
    print(f"Training rows after label filtering: {len(y):,} | classes: {sorted(y.unique())}")

    # Train/test split
    stratify = y if y.nunique() > 1 else None
    X_train, X_test, y_train, y_test = train_test_split(
        X_text, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=stratify
    )

    # Pipeline
    tfidf = TfidfVectorizer(lowercase=True, stop_words="english",
                            ngram_range=NGRAM_RANGE, max_features=MAX_FEATURES)
    clf = LogisticRegression(max_iter=1000, class_weight=("balanced" if USE_CLASS_WEIGHT_BALANCED else None), solver="lbfgs", multi_class="auto")
    pipe = Pipeline([("tfidf", tfidf), ("clf", clf)])

    # Fit
    pipe.fit(X_train, y_train)

    # Predict
    y_pred = pipe.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    overall = {
        "accuracy": accuracy_score(y_test, y_pred),
        "macro_f1": f1_score(y_test, y_pred, average="macro"),
        "weighted_f1": f1_score(y_test, y_pred, average="weighted"),
    }
    print("Overall:", overall)
    print(pd.DataFrame(report).T)

    # Confusion matrix
    labels = sorted(y.unique())
    cm = confusion_matrix(y_test, y_pred, labels=labels)
    fig_cm = FIGS / f"{ARTI_BASE}_confusion_matrix.png"
    plot_confusion(cm, labels, "Confusion matrix (TF-IDF + LogReg)", fig_cm)

    # Probabilities and prediction export
    proba = None
    try:
        proba = pipe.predict_proba(X_test)
        max_p = proba.max(axis=1)
    except Exception:
        max_p = np.ones(len(y_pred))

    pred_df = pd.DataFrame({
        "text": X_test.values,
        "y_true": y_test.values,
        "y_pred": y_pred,
        "confidence": max_p
    })
    # Attach ids and timestamps if present
    for col in ["tweet_id","username","created_at"]:
        if col in df.columns:
            pred_df[col] = df.loc[X_test.index, col].values

    preds_path = REPORTS / f"{ARTI_BASE}_supervised_predictions.csv"
    pred_df.to_csv(preds_path, index=False, encoding="utf-8")
    print("Saved predictions:", preds_path)

    # Save model
    model_path = MODELS / f"{ARTI_BASE}_tfidf_logreg.joblib"
    joblib.dump(pipe, model_path)
    print("Saved model:", model_path)

    # Save metrics
    metrics = {
        "run_id": RUN_ID,
        "mode": "supervised",
        "input": str(proc_path.relative_to(PROJ)),
        "n_train": int(len(X_train)),
        "n_test": int(len(X_test)),
        "labels": labels,
        "overall": overall,
        "classification_report": report,
        "params": {
            "test_size": TEST_SIZE,
            "random_state": RANDOM_STATE,
            "ngram_range": NGRAM_RANGE,
            "max_features": MAX_FEATURES,
            "class_weight": "balanced" if USE_CLASS_WEIGHT_BALANCED else None
        }
    }
    metrics_path = REPORTS / f"{ARTI_BASE}_supervised_metrics.json"
    with open(metrics_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2)
    print("Saved metrics:", metrics_path)

    # Feature importance (per class)
    try:
        vec = pipe.named_steps["tfidf"]
        clf = pipe.named_steps["clf"]
        vocab = np.array(vec.get_feature_names_out())

        coef = clf.coef_
        classes = clf.classes_
        top_k = 30

        for i, cls in enumerate(classes):
            # top positive weights for class cls
            order = np.argsort(coef[i])[::-1][:top_k]
            terms = pd.DataFrame({"term": vocab[order], "weight": coef[i][order]})
            out = REPORTS / f"{ARTI_BASE}_top_terms_class_{cls}.csv"
            terms.to_csv(out, index=False, encoding="utf-8")
            print(f"Saved top terms for class {cls} -> {out}")
    except Exception as e:
        print("Skipping feature importance:", e)

# 7) Unsupervised baseline (VADER)

In [ ]:
if resolved_mode == "unsupervised":
    # Optionally filter to English for VADER
    df_v = df.copy()
    if VADER_ONLY_ENGLISH and "lang" in df_v.columns:
        before = len(df_v)
        df_v = df_v[df_v["lang"] == "en"]
        print(f"VADER: kept English only: {len(df_v):,}/{before:,}")

    # Prepare text
    texts = df_v[TEXT_COL].fillna("").astype(str).tolist()

    # VADER scoring
    try:
        import nltk
        nltk.download("vader_lexicon", quiet=True)
        from nltk.sentiment import SentimentIntensityAnalyzer
        sia = SentimentIntensityAnalyzer()
    except Exception as e:
        raise RuntimeError("Install nltk to use VADER: pip install nltk") from e

    def comp_to_label(c):
        if c >= VADER_POS_THRESHOLD: return "pos"
        if c <= VADER_NEG_THRESHOLD: return "neg"
        return "neu"

    compounds = [sia.polarity_scores(t)["compound"] for t in texts]
    vader_labels = [comp_to_label(c) for c in compounds]

    # Build output frame
    out_df = df_v.copy()
    out_df["vader_compound"] = compounds
    out_df["vader_label"] = vader_labels

    # If ground-truth labels exist, evaluate
    eval_report = {}
    if LABEL_COLUMN in out_df.columns:
        gt = out_df[LABEL_COLUMN].map(standardize_label)
        mask = gt.notna()
        if mask.sum() >= 20 and gt[mask].nunique() >= 2:
            y_true = gt[mask]
            y_pred = pd.Series(out_df.loc[mask, "vader_label"].values, index=y_true.index)
            rep = classification_report(y_true, y_pred, output_dict=True)
            eval_report = {
                "accuracy": accuracy_score(y_true, y_pred),
                "macro_f1": f1_score(y_true, y_pred, average="macro"),
                "weighted_f1": f1_score(y_true, y_pred, average="weighted"),
                "classification_report": rep
            }
            print("VADER vs labels (where available):")
            print(pd.DataFrame(rep).T)

    # Aggregates
    label_counts = out_df["vader_label"].value_counts().to_dict()
    avg_compound = float(np.mean(out_df["vader_compound"])) if len(out_df) else None

    print("Label distribution:", label_counts)
    print("Average compound:", avg_compound)

    # Save artifacts
    out_parquet = REPORTS / f"{ARTI_BASE}_vader_scores.parquet"
    out_csv = REPORTS / f"{ARTI_BASE}_vader_scores_sample.csv"
    out_df.to_parquet(out_parquet, index=False)
    out_df.head(2000).to_csv(out_csv, index=False, encoding="utf-8")
    print("Saved:", out_parquet)
    print("Saved sample CSV:", out_csv)

    # Time trend plot
    if "created_at" in out_df.columns and out_df["created_at"].notna().any():
        daily = (out_df.set_index("created_at")["vader_compound"]
                 .resample("D").mean().reset_index())
        plt.figure(figsize=(8,3))
        sns.lineplot(data=daily, x="created_at", y="vader_compound")
        plt.title("VADER: average compound per day")
        plt.tight_layout()
        fig_path = FIGS / f"{ARTI_BASE}_vader_daily.png"
        plt.savefig(fig_path, dpi=150)
        plt.show()
        print("Saved:", fig_path)

    # Save run summary
    summary = {
        "run_id": RUN_ID,
        "mode": "unsupervised",
        "input": str(proc_path.relative_to(PROJ)),
        "rows_scored": int(len(out_df)),
        "text_col": TEXT_COL,
        "vader_thresholds": {"pos": VADER_POS_THRESHOLD, "neg": VADER_NEG_THRESHOLD},
        "label_distribution": label_counts,
        "avg_compound": avg_compound,
        "evaluation_if_labels": eval_report
    }
    summary_path = REPORTS / f"{ARTI_BASE}_vader_summary.json"
    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2)
    print("Saved summary:", summary_path)

# 8) Transformer sentiment inference

In [ ]:
if USE_TRANSFORMER:
    try:
        from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
        import torch
    except Exception as e:
        raise RuntimeError("Install transformers/torch to run this block. See the pip cell above.") from e

    model_name = TRANSFORMER_MODEL
    print("Loading model:", model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = 0 if torch.cuda.is_available() else -1
    pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer,
                                      device=device, truncation=True, top_k=None, return_all_scores=True)

    df_t = df.copy()
    texts = df_t[TEXT_COL].fillna("").astype(str).tolist()

    # Batch inference
    preds_all = []
    for i in range(0, len(texts), BATCH_SIZE):
        batch = texts[i:i+BATCH_SIZE]
        preds_all.extend(pipe(batch))

    # Convert outputs to scores and labels
    # CardiffNLP label order typically: 0=negative, 1=neutral, 2=positive
    def parse_scores(entry):
        try:
            scores = {d["label"].lower(): float(d["score"]) for d in entry}
            # Normalize common label names
            mapping = {}
            for k, v in scores.items():
                if "neg" in k: mapping["neg"] = v
                elif "neu" in k: mapping["neu"] = v
                elif "pos" in k: mapping["pos"] = v
            if not mapping:
                # Fall back to max label if not matched
                best = max(scores.items(), key=lambda x: x[1])[0].lower()
                if "neg" in best: mapping = {"neg": 1.0, "neu": 0.0, "pos": 0.0}
                elif "neu" in best: mapping = {"neg": 0.0, "neu": 1.0, "pos": 0.0}
                else: mapping = {"neg": 0.0, "neu": 0.0, "pos": 1.0}
            # Pred label
            pred = max(mapping.items(), key=lambda x: x[1])[0]
            return mapping.get("neg", np.nan), mapping.get("neu", np.nan), mapping.get("pos", np.nan), pred
        except Exception:
            return np.nan, np.nan, np.nan, np.nan

    neg_scores, neu_scores, pos_scores, preds = [], [], [], []
    for entry in preds_all:
        n, u, p, lbl = parse_scores(entry)
        neg_scores.append(n); neu_scores.append(u); pos_scores.append(p); preds.append(lbl)

    out_df = df_t.copy()
    out_df["hf_neg"] = neg_scores
    out_df["hf_neu"] = neu_scores
    out_df["hf_pos"] = pos_scores
    out_df["hf_label"] = preds

    # Evaluate if labels exist
    eval_report = {}
    if LABEL_COLUMN in out_df.columns:
        gt = out_df[LABEL_COLUMN].map(standardize_label)
        mask = gt.notna()
        if mask.sum() >= 20 and gt[mask].nunique() >= 2:
            y_true = gt[mask]
            y_pred = out_df.loc[mask, "hf_label"]
            rep = classification_report(y_true, y_pred, output_dict=True)
            eval_report = {
                "accuracy": accuracy_score(y_true, y_pred),
                "macro_f1": f1_score(y_true, y_pred, average="macro"),
                "weighted_f1": f1_score(y_true, y_pred, average="weighted"),
                "classification_report": rep
            }
            print("Transformer vs labels:")
            print(pd.DataFrame(rep).T)

    out_parquet = REPORTS / f"{ARTI_BASE}_hf_{Path(model_name).name}_scores.parquet"
    out_csv = REPORTS / f"{ARTI_BASE}_hf_{Path(model_name).name}_scores_sample.csv"
    out_df.to_parquet(out_parquet, index=False)
    out_df.head(2000).to_csv(out_csv, index=False, encoding="utf-8")
    print("Saved:", out_parquet)
    print("Saved sample CSV:", out_csv)

    summary = {
        "run_id": RUN_ID,
        "mode": "transformer_inference",
        "model": model_name,
        "input": str(proc_path.relative_to(PROJ)),
        "rows_scored": int(len(out_df)),
        "evaluation_if_labels": eval_report
    }
    summary_path = REPORTS / f"{ARTI_BASE}_hf_{Path(model_name).name}_summary.json"
    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2)
    print("Saved summary:", summary_path)